In [24]:
import numpy as np
import pandas as pd

In [36]:
credits= pd.read_csv('../Datos/credits.csv')
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [6]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [37]:
credits[credits.duplicated(keep=False)].sort_values(by='id') 
credits.drop_duplicates(inplace=True)

In [42]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [45]:
credits.reset_index(drop=True, inplace=True)

In [39]:
#con ast convierto las cadenas de string a diccionarios
import ast

credits["cast"] = credits["cast"].apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else np.nan)
credits["crew"] = credits["crew"].apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else np.nan)

In [46]:
#ahora itero todo, guardo cada df en la lista que luego concatenare al nuevo df
datos = []
casts = credits["cast"]
id_original = credits["id"]

for idx, cast in enumerate(casts): 
    for d in cast:
        d["id"] = id_original[idx] # Agrego la columna "id" con el ID de la película
    df = pd.DataFrame.from_records(cast)
    datos.append(df)

cast_df = pd.concat(datos, ignore_index=True)


In [47]:
#crew
datos_crew= []
crews= credits["crew"]
id_original = credits["id"]

for idx, crew in enumerate(crews):
    for d in crew:
       d["id"] = id_original[idx] 
    df = pd.DataFrame.from_records(crew)
    datos_crew.append(df)
    
crew_df = pd.concat(datos_crew, ignore_index=True)

In [23]:
#exporto los df creados a csv
cast_df.to_csv('cast.csv')
crew_df.to_csv('crew.csv')